In [1]:
from WindPy import w
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

import talib as tb
import alphalens as al
from sqlalchemy import create_engine, Table, Column, Integer
from sklearn.preprocessing import StandardScaler
import datetime


_ = w.start()
scaler = StandardScaler()

pd.set_option('display.max_rows', None)       # 显示所有行
pd.set_option('display.max_columns', None)    # 显示所有列

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
# 提取申万一级行业
# engine = create_engine("mysql+pymysql://root:@localhost:3306/meta_data?charset=utf8mb4")
list_indicator = ["close","volume","amt","pct_chg","high","low","open","turn"]
list_industry = ["801010.SI","801030.SI","801040.SI","801050.SI","801080.SI","801110.SI","801120.SI","801130.SI","801140.SI","801150.SI","801160.SI","801170.SI","801180.SI","801200.SI","801210.SI","801710.SI","801720.SI","801730.SI","801740.SI","801750.SI","801760.SI","801770.SI","801780.SI","801790.SI","801880.SI","801890.SI","801950.SI","801960.SI","801970.SI","801980.SI"]  # 去除综合
list_indicator_consencus_expection = ["west_netprofit_CAGR","west_eps_FY1","west_netprofit_YOY","west_avgroe_FY1","estpeg_FY1"]
list_indicator_using = ['open','close','amt','volume','turn','west_avgroe_FY1']  # 因子筛选后在用的indicator

# Func and Ini

In [3]:
def calculate_second_order_mom(close_prices, window1, window2, window3):
    
    def ewma(series, span): 
        return series.ewm(span=span, adjust=False).mean()
    
    close_series = pd.Series(close_prices)  # convert to series
    mean_window1 = close_series.rolling(window=window1).mean()  # calculate mean value
    fraction = (close_series - mean_window1)/ mean_window1  # Calculate the fraction

    delayed_fraction = fraction.shift(window2)  # delay (shift) operation
    
    result = ewma(fraction - delayed_fraction, span=window3)  # Calculate the final result using EWM
    
    return result

def calculate_momentum_spread(close_price,window1,window2):
    
    close_series = pd.Series(close_price)  # convert to series
    
    mean_window1 = close_series.rolling(window=window1).mean()  # calculate mean value in short term 
    long_fraction = (close_series - mean_window1)/ mean_window1  # calculate the fraction in short term
    
    mean_window2 = close_price.rolling(window=window2).mean()  # calculate mean value in long term
    short_fraction = (close_series - mean_window2)/ mean_window2  # calculate the fraction in short term
    
    momentum_spread = long_fraction-short_fraction
    
    return momentum_spread

def get_factor_dataframe_for_alphalens(dataframe):
    
    # 初始化数据格式
    dataframe.index = pd.to_datetime(dataframe.index)
    dataframe.reset_index(names=['date'],inplace=True)
    dataframe.dropna(inplace=True)

    # 调整df结构
    dataframe_:pd.DataFrame = dataframe.melt(var_name='asset',id_vars=['date'],value_name='factor_value')
    dataframe_.set_index(['date','asset'],inplace=True)
    
    return dataframe_

def ini_industry_value(dataframe):
    
    dataframe.index = pd.to_datetime(dataframe.index) # ini date.index
    dataframe = dataframe.shift(-1)# 错位对齐
    dataframe.dropna(inplace=True)
    
    return dataframe

def calculate_turnover_rate(turnover,window1,window2):  # window1>window2
    
    turnover_series = pd.Series(turnover)  # convert to series

    mean_window1 = turnover_series.rolling(window1).mean()  # calculate mean turnover for window1
    mean_window2 = turnover_series.rolling(window2).mean()  # calculate mean turnover for window2
    
    turnover_rate = mean_window1/mean_window2
    
    return turnover_rate

def calculate_long_short_position(df,column_name,window1):
    
    # alphalens会自动把0和nan都drop掉，认为0和nan都没用，所以需要先判断高、低、收的价格确保不相等
    
    epsilon = 1e-9
    
    df['LOW'] = np.where(np.abs(df['CLOSE']-df['LOW'])<epsilon,df['LOW']-np.random.rand(),df['LOW'])
    df['HIGH'] = df.apply(lambda row:row['HIGH']+np.random.rand() if np.abs(row['HIGH']-row['CLOSE'])<epsilon else row['HIGH'],axis=1)


    long = df['CLOSE'] - df['LOW']
    short = df['HIGH'] - df['CLOSE']
    
    df[column_name] = long.rolling(window1).sum()/short.rolling(window1).sum()  # 计算long/short在窗口期的和
    
    return df[column_name]

def calculate_long_short_change(df,column_name,window1,window2):
    
    def ewma(series, span): 
        return series.ewm(span=span, adjust=False).mean()
    
    long = df['CLOSE'] - df['LOW']
    short = df['HIGH'] - df['CLOSE']
    spread = df['HIGH']- df['LOW']
    spread = spread.replace(0, 1e-10)  # 替换分母的0值

    volume_long_short = df['VOLUME']*((long-short)/spread)
    

    long_fraction = ewma(series=volume_long_short,span=window1)
    short_fraction = ewma(series=volume_long_short,span=window2)
    
    df[column_name] =  short_fraction - long_fraction
    
    return df[column_name]

def calculate_close_volume_divergence(df, window,column_name):
    
    def rank_series(series):
        return series.rank()

    def rolling_covariance(x, y, window):
        return x.rolling(window).cov(y)  # 窗口window下x和y的协方差
    
    # 计算 Close 和 Volume 的排名
    rank_close = rank_series(df['CLOSE'])
    rank_volume = rank_series(df['VOLUME'])
    
    # 计算排名后的协方差
    cov = rolling_covariance(rank_close, rank_volume, window)
    
    # 对协方差进行排名
    df[column_name] = rank_series(cov)
    
    return df[column_name]

def calculate_close_volume_divergence_corr(df,column_name,window):
    
    def rolling_correlation(x, y, window):
        return x.rolling(window).corr(y)  # 窗口window下x和y的协方差
    
    corr = rolling_correlation(df['CLOSE'], df['VOLUME'], window)
    
    df[column_name] = corr
    
    return df[column_name]

def calculate_first_order_diff_divergence(df,column_name,window):
    
    def rank_series(series):
        return series.rank()
    
    
    def rolling_correlation(x, y, window):
        return x.rolling(window).corr(y)  # 窗口window下x和y的协方差
    
    # 计算 Close 和 Volume 的排名
    first_order_diff_close = rank_series(df['CLOSE']/df['CLOSE'].shift(1)-1)
    first_order_diff_volume = rank_series(df['VOLUME']/df['VOLUME'].shift(1)-1)
    
    corr = rolling_correlation(first_order_diff_close, first_order_diff_volume, window)

    
    df[column_name] = corr
    
    return df[column_name]

def calculate_volume_HDL_diff_divergence(df,column_name,window):
    
    def rank_series(series):
        return series.rank()
    
    def rolling_correlation(x, y, window):
        return x.rolling(window).corr(y)  # 窗口window下x和y的协方差
    
    # 计算 hdl 和 Volume 的排名
    rank_HDL = rank_series(df['HIGH']/df['LOW']-1)
    rank_volume = rank_series(df['VOLUME']/df['VOLUME'].shift(1)-1)
    
    corr = rolling_correlation(rank_volume, rank_HDL, window)
    
    df[column_name] = corr
    
    return df[column_name]

def get_factor_value_extracted(isExtract=False,**params):
    
    df = params['df']
    date = params['date']
    
    if not isExtract:
        return df
    else:
        df = df[df.index>date]
        return df
    
def day2week(df):
    df.index = pd.to_datetime(df.index)
    df_ = df.resample('W').last()  # 周度假期导致交易日丢失
    return df_



def get_unconstant_variables(df):
    # 创建一个布尔掩码，标记与上一行相同的行
    mask = (df == df.shift(1)).all(axis=1)
    # 对与上一行相同的行进行加1操作
    random_noise = (np.random.rand(*df.shape) * 2 - 1)  # randm.rand 生成*df.shape的随机数
    df[mask] = df[mask] + random_noise[mask]
    return df

def get_rolling_ic(df,rollingwindow):
    ic = df.groupby(ret.index.get_level_values('date')).apply(lambda x: x['factor'].corr(x['1D'], method='spearman'))
    ic_rolling = ic.rolling(rollingwindow).mean().dropna()
    return ic_rolling

def get_rolling_ic_ir(df,rollingwindow1,rollingwindow2):
    
    ic = df.groupby(ret.index.get_level_values('date')).apply(lambda x: x['factor'].corr(x['1D'], method='spearman'))
    ic_rollinglling_mean = ic.rolling(rollingwindow1).apply(lambda roll: roll.mean())
    ic_rollinglling_std = ic.rolling(rollingwindow2).apply(lambda roll: roll.std())

    ic_ir = ic_rollinglling_mean/ic_rollinglling_std
    return ic_ir


def day2month(df):
    df.index = pd.to_datetime(df.index)
    df_ = df.resample('M').last()  # 周度假期导致交易日丢失
    return df_

def calculate_ic(factor,window):
    ic = industry_ret.apply(lambda row:row.corr(factor.loc[row.name],method='spearman'),axis=1).rolling(window).mean().dropna()
    return ic

def calculate_icir(factor,rollingwindow1,rollingwindow2):
    
    ic = industry_ret.apply(lambda row:row.corr(factor.loc[row.name],method='spearman'),axis=1).rolling(rollingwindow1).mean().dropna()
    
    ic_rollinglling_mean = ic.rolling(rollingwindow1).apply(lambda roll: roll.mean())
    ic_rollinglling_std = ic.rolling(rollingwindow2).apply(lambda roll: roll.std())
    ic_ir = ic_rollinglling_mean/ic_rollinglling_std
    
    return ic_ir

def zscroe_cross_sectional_data(df):
    return df.apply(lambda row:(row-row.mean())/row.std(),axis=1)

def calculate_quantile_groups(df, num_quantiles):
    # 按日期分组并计算每个日期截面的分位数分组
    quantile_groups = df.apply(lambda row: pd.qcut(row.rank(method='first'), num_quantiles, labels=False) + 1,axis=1)
    return quantile_groups

# Update

In [14]:
print('***'*39)

*********************************************************************************************************************


In [9]:
fdate_meta,tdate_meta = '2024-08-01','2024-08-30'  # 月频更新，输入月第一个工作日，月最后一个工作日
today = datetime.datetime.today().date()

In [10]:
engine = create_engine("mysql+pymysql://root:@localhost:3306/industry_rotation_meta_data?charset=utf8mb4")

## creat by sql

创建数据库用的，平时用不到

### indicator by metric

In [6]:
for ind in list_indicator_using:
    temp = pd.read_pickle('./db/{}.pkl'.format(ind+"_SI_d"))
    temp.to_sql(name='{}_si_d'.format(ind).lower(),con=engine,if_exists='replace',index=True,index_label='Date')

### indicator by industries

In [41]:
for ind in list_industry:
    temp = pd.read_pickle('./db/{}.pkl'.format(ind+"._d"))
    temp.to_sql(name='{}_d'.format(ind).lower(),con=engine,if_exists='replace',index=True,index_label='Date')

## update by sql

In [8]:
# indicator分类，更新数据
for indr in list_indicator_using:
    errorcode,__  = w.wsd("801010.SI,801030.SI,801040.SI,801050.SI,801080.SI,801110.SI,801120.SI,801130.SI,801140.SI,801150.SI,801160.SI,801170.SI,801180.SI,801200.SI,801210.SI,801710.SI,801720.SI,801730.SI,801740.SI,801750.SI,801760.SI,801770.SI,801780.SI,801790.SI,801880.SI,801890.SI,801950.SI,801960.SI,801970.SI,801980.SI", indr, fdate_meta, tdate_meta, "Days=Alldays;Fill=Previous",usedf=True)  #剔除综合   
    __.to_sql(name='{}_si_d'.format(indr).lower(),con=engine,if_exists='append',index=True,index_label='Date')

NameError: name 'fdate_meta' is not defined

sql table raise erro when '.' is part of the name

In [13]:
#分industry分类，更新数据
for ind in list_industry:
    errorcode,_ = w.wsd(ind, "close,volume,amt,pct_chg,high,low,open", fdate_meta, tdate_meta, "Days=Alldays;Fill=Previous",usedf=True)
    _.to_sql(name='{}_d'.format(ind).lower().replace('.','_'),con=engine,if_exists='append',index=True,index_label='Date')

# meta_data


In [11]:
# open
industry_open_real:pd.DataFrame = pd.read_sql('SELECT * FROM open_si_d',engine,index_col='Date')
# 清理industry_close数据结构
industry_open_real = get_unconstant_variables(industry_open_real)  # df中不可出现constant
industry_open_real_ = day2month(industry_open_real)

In [12]:
# 提取 data_by_industries
industry_close:pd.DataFrame = pd.read_sql('SELECT * FROM close_si_d',engine,index_col='Date')
industry_amt:pd.DataFrame = pd.read_sql('SELECT * FROM amt_si_d',engine,index_col='Date')
industry_volume:pd.DataFrame = pd.read_sql('SELECT * FROM volume_si_d',engine,index_col='Date')
industry_turn:pd.DataFrame = pd.read_sql('SELECT * FROM turn_si_d',engine,index_col='Date')
roe_fy1:pd.DataFrame = pd.read_sql('SELECT * FROM west_avgroe_fy1_si_d',engine,index_col='Date')
# ini
industry_close.index = pd.to_datetime(industry_close.index)
industry_amt.index = pd.to_datetime(industry_amt.index)
industry_volume.index = pd.to_datetime(industry_volume.index)
industry_turn.index = pd.to_datetime(industry_turn.index)

##  second_oredr_mom

In [17]:
# calculate factor value
second_order_mom = -industry_close.apply(calculate_second_order_mom,window1=10,window2=20,window3=20)
# 初始化数据格式e
second_order_mom.index = pd.to_datetime(second_order_mom.index)
second_order_mom = second_order_mom.resample("M").last() # 上个月因子值对应下个月开盘价

## std_volume

In [18]:
std_volume:pd.DataFrame = -industry_volume.rolling(60).std().resample('M').last()

## turnover_rate

In [19]:
turnover_rate:pd.DataFrame=industry_turn.apply(calculate_turnover_rate,window1=80,window2=5)  # 计算turnover_rate
# 初始化turnover_rate，转化为月度数据
turnover_rate = turnover_rate.resample('M').last()
turnover_rate = get_factor_value_extracted(isExtract=False,df=turnover_rate,date="2021-01-01")

## long_short_position

In [24]:
print('***'*39)

*********************************************************************************************************************


In [20]:
long_short_position :pd.DataFrame=pd.DataFrame()  # 创建空df储存industry因子值
for ind in list_industry:
    temp = pd.read_sql('SELECT * FROM {}'.format(ind+'_d').lower().replace('.','_'),engine,index_col='Date')
    temp_ = - calculate_long_short_position(df=temp,column_name=ind,window1=15)
    long_short_position = pd.concat([long_short_position,temp_],axis=1)
# 初始化factor_value，转化为月度数据
long_short_position.index = pd.to_datetime(long_short_position.index)
long_short_position = long_short_position.resample('M').last()  # 上个月因子值对应下个月开盘价

## long_short_change

In [21]:
long_short_change:pd.DataFrame=pd.DataFrame()
for ind in list_industry:
    temp = pd.read_sql('SELECT * FROM {}'.format(ind+'_d').lower().replace('.','_'),engine,index_col='Date')
    temp_ = - calculate_long_short_change(df=temp,column_name=ind,window1=120,window2=15)
    long_short_change = pd.concat([long_short_change,temp_],axis=1)
# 初始化factor_value，转化为月度数据
long_short_change.index = pd.to_datetime(long_short_change.index)
long_short_change = long_short_change.resample('M').last()  

## close_volume_divergence

In [22]:
close_volume_divergence:pd.DataFrame=pd.DataFrame()
for ind in list_industry:
    temp = pd.read_sql('SELECT * FROM {}'.format(ind+'_d').lower().replace('.','_'),engine,index_col='Date')
    temp_ = - calculate_close_volume_divergence(df=temp,window=60,column_name=ind)
    close_volume_divergence = pd.concat([close_volume_divergence,temp_],axis=1)
# 初始化factor_value
close_volume_divergence.index = pd.to_datetime(close_volume_divergence.index)
close_volume_divergence = close_volume_divergence.resample('M').last()

## close_volume_divergence_corr

In [23]:
close_volume_divergence_corr:pd.DataFrame=pd.DataFrame()
for ind in list_industry:
    temp = pd.read_sql('SELECT * FROM {}'.format(ind+'_d').lower().replace('.','_'),engine,index_col='Date')
    temp_ = - calculate_close_volume_divergence_corr(df=temp,window=60,column_name=ind)
    close_volume_divergence_corr = pd.concat([close_volume_divergence_corr,temp_],axis=1)
# 初始化factor_value
close_volume_divergence_corr.index = pd.to_datetime(close_volume_divergence_corr.index)
close_volume_divergence_corr = close_volume_divergence_corr.resample('M').last()

## volume_HDL_diff_divergence

In [24]:
volume_HDL_diff_divergence:pd.DataFrame=pd.DataFrame()
for ind in list_industry:
    temp = pd.read_sql('SELECT * FROM {}'.format(ind+'_d').lower().replace('.','_'),engine,index_col='Date')
    temp_ = calculate_volume_HDL_diff_divergence(df=temp,window=10,column_name=ind)
    volume_HDL_diff_divergence = pd.concat([volume_HDL_diff_divergence,temp_],axis=1)
# 初始化factor_value
volume_HDL_diff_divergence.index = pd.to_datetime(volume_HDL_diff_divergence.index)
volume_HDL_diff_divergence = volume_HDL_diff_divergence.resample('M').last()

## roe_fy1

In [25]:
# scaler zscroe
scaler = StandardScaler()
for col in roe_fy1.columns:
    roe_fy1[col] = scaler.fit_transform(roe_fy1[[col]])
# 初始化factor_value
roe_fy1.index = pd.to_datetime(roe_fy1.index)
roe_fy1 = - roe_fy1.resample('M').last()

# get_ic

- 这里的问题是，ic值总是滞后一个月，因为ic的计算需要下个月的收益率，即下下个月的开盘价
- 手动计算的ic和alphalensic有微小区别可能是因为两者保留的小数位不同

In [54]:
print('***'*39)

*********************************************************************************************************************


In [34]:
dic_factor = {
    
    "second_order_mom":second_order_mom,
    "std_volume":std_volume,
    "turnover_rate":turnover_rate,
    "long_short_position":long_short_position,
#     "long_short_change":long_short_change,
#     "close_volume_divergence":close_volume_divergence,
    "close_volume_divergence_corr":close_volume_divergence_corr,
    "volume_HDL_diff_divergence":volume_HDL_diff_divergence,
#     "roe_fy1":roe_fy1
}
dic_factor_ = {k:zscroe_cross_sectional_data(v.dropna()) for k,v in dic_factor.items()}  # 对csd标准化处理

In [40]:
industry_ret = industry_open_real_.pct_change(1).shift(-2).dropna().loc['2017-01-31':]

In [41]:
dic_ic = {}  # 存储不同因子ic值
for k,v in dic_factor.items():
    dic_ic[k] = calculate_ic(v,window=3)

In [42]:
dic_factor_ = {k:dic_factor_[k].loc[dic_ic[k].index.min():today] for k in dic_factor_.keys()}# 截取数据 sdate=ic sdate;edate=last month

# get_composite_factor_icw

我们因子合成的逻辑和前面回测有所不同：
- 对于单个因子，我们对截面进行zscore得到因子值（这一步alphalens没做，但对结果没影响）
- 对于因子复合阶段，确实是在截面zscore相对合理一些，这里就是这么做的

In [47]:
quantile_groups_df.to_excel('3.xlsx')

In [27]:
top_rank_industry

801170.SI    5
801730.SI    5
801740.SI    5
801750.SI    5
801790.SI    5
801890.SI    5
Name: 2024-07-31 00:00:00, dtype: int64

In [29]:
print('***'*39)

*********************************************************************************************************************


In [43]:
# get clean indicator values
dic_indicator = dic_factor_

In [44]:
dic_factor_weighted = {}  # empty dic for weighted factor values
# 计算icw合并因子
for key,value in dic_indicator.items():# 获取ic*factor_value dic
    dic_factor_weighted[key] = value.apply(lambda row: row * dic_ic[key].loc[:row.name][-1],axis=1)
df_composite_factor = sum(dic_factor_weighted[key] for key in dic_factor_weighted.keys()) #  因子值相加

In [45]:
#calculate_quantile_groups
df_composite_factor.dropna(inplace=True)
quantile_groups_df = calculate_quantile_groups(df_composite_factor, num_quantiles=5)

In [26]:
# 获取排名前5的行业
top_rank_industry = quantile_groups_df.iloc[-1][quantile_groups_df.iloc[-1]==5]